# MA934 Numerical Methods - Workbook 3

If you haven't already done so, install the DualNumbers Julia package. It is a good idea to update all your packages first. The commands are

>Pkg.update()

>Pkg.add("DualNumbers")

but you only need to run them once. 

In [ ]:
#Pkg.update()
#Pkg.add("DualNumbers")
using Plots
using DualNumbers

## Question 1: Numerical differentiation

**1))** Derive a finite difference formula for the derivative of a function $f$ at a point $x$ using the 3-point stencil $(x, x+h, x+2h)$ and state the order of the approximation error in terms of $h$.

**2)** Write a formula for the derivative, $f^\prime(x)$, of the function

$$f(x) = \sin(\exp(x)) $$

and evaluate it at $x=1$.

**3)** Use your finite difference formula to approximate the value of $f^\prime(1)$ for values of $h$ decreasing from $2^{-1}$ to $2^{-30}$ in powers of $2$. Plot the error as a function of $h$ and verify the theoretically predicted scaling of the error with $h$. What is the best relative error you can achieve?

**4)** Read the examples at https://github.com/JuliaDiff/DualNumbers.jl. Define a dual number $x = 1+\epsilon$ and use it to evaluate $f^\prime(1)$. Verify that the answer is accurate to within machine precision.

In [102]:
f(x) = sin(exp(x))
f_theoretical(x) = cos(exp(x)) * exp(x)
f_approx(x,h) = (3*f(x+2h)-4*f(x+h)+f(x)). * h.^(-1) * (-0.5) 
temp(x,h) = (3*f(x+2h)-4*f(x+h)+f(x))

LoadError: syntax: extra token "h" after end of expression

In [87]:
h = 2.0.^(-collect(1:30))  

30-element Array{Float64,1}:
 0.5        
 0.25       
 0.125      
 0.0625     
 0.03125    
 0.015625   
 0.0078125  
 0.00390625 
 0.00195313 
 0.000976563
 0.000488281
 0.000244141
 0.00012207 
 ⋮          
 1.90735e-6 
 9.53674e-7 
 4.76837e-7 
 2.38419e-7 
 1.19209e-7 
 5.96046e-8 
 2.98023e-8 
 1.49012e-8 
 7.45058e-9 
 3.72529e-9 
 1.86265e-9 
 9.31323e-10

In [92]:
z = temp(1,h)

30-element Array{Float64,1}:
  6.98637    
 -1.14284    
 -0.859739   
 -0.387057   
 -0.175549   
 -0.0827351  
 -0.0400589  
 -0.0196972  
 -0.00976502 
 -0.00486154 
 -0.00242552 
 -0.00121145 
 -0.000605394
  ⋮          
 -9.45423e-6 
 -4.7271e-6  
 -2.36354e-6 
 -1.18177e-6 
 -5.90885e-7 
 -2.95442e-7 
 -1.47721e-7 
 -7.38606e-8 
 -3.69303e-8 
 -1.84651e-8 
 -9.23257e-9 
 -4.61629e-9 

In [94]:
temp(1,h).*h.^(-1) 

30-element Array{Float64,1}:
 13.9727 
 -4.57138
 -6.87791
 -6.19291
 -5.61756
 -5.29504
 -5.12754
 -5.0425 
 -4.99969
 -4.97822
 -4.96746
 -4.96208
 -4.95939
  ⋮      
 -4.95674
 -4.95672
 -4.95671
 -4.9567 
 -4.9567 
 -4.9567 
 -4.9567 
 -4.9567 
 -4.9567 
 -4.9567 
 -4.9567 
 -4.9567 

In [101]:
f_approx(1.0,0.00000001)

2.478349972467875

## Question 2: Finding roots

**1)** Referring to the function, $f(x)$, defined above, find the roots of the equation

$$ f(x) = 0$$

in the interval $0<x<2$.

**2)** Implement the bracketing and bisection method to find one of the roots numerically. Measure the error at each iteration of the algorithm and demonstrate that the error decreases exponentially as a function of the number of iterations. To how many digits of precision can you approximate the root?

**3)** Perform the same measurements for the Newton Raphson method and show that the error decreases faster than exponentially as a function of the number of iterations.

## Question 3: Finding minima

**1)** The function $f(x)$ above has a single minimum in the interval $0<x<2$. Find its location analytically.

**2)** Implement the Golden section search to find the location of this minimum numerically. Plot the error as a function of the number of iterations. To how many digits of precision can you approximate the location of the minimum?

**3)** To understand your empirical findings, use Taylor's Theorem to show that near a minimum, $x_*$, of f(x),

$$f(x) \approx f(x_*)\left( 1+ \frac{f^{\prime\prime}(x_*)}{2\,f(x_*)}\,(x-x_*)^2\right). $$
Show that in order for a computer to distinguish between $f(x)$ and $f(x_*)$ we must have

$$ \left| x-x_*\right| > \sqrt{\epsilon_m}\,\sqrt{\left|\frac{2\,f(x_*)}{f^{\prime\prime}(x_*)}\right|}$$

thus limiting the precision with which the location of a minimum can be determined.